In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import re
import pandas as pd
import numpy as np

from torch.utils.data import DataLoader

import data_handling
import extract_values

In [3]:
from PIL import Image
import requests
from transformers import AutoProcessor, BlipForImageTextRetrieval

c:\Users\sunny\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#load CLIP
from transformers import CLIPProcessor, CLIPModel
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

c:\Users\sunny\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\sunny\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
c:\Users\sunny\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
c:\Users\sunny\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprec

In [3]:
# load FLAVA
from transformers import FlavaProcessor, FlavaForPreTraining
flava_model = FlavaForPreTraining.from_pretrained("facebook/flava-full")
flava_processor = FlavaProcessor.from_pretrained("facebook/flava-full", return_codebook_pixels=True)

c:\Users\sunny\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\sunny\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
c:\Users\sunny\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
c:\Users\sunny\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprec

In [20]:
# load blip

blip_model = BlipForImageTextRetrieval.from_pretrained("Salesforce/blip-itm-base-coco")
blip_processor = AutoProcessor.from_pretrained("Salesforce/blip-itm-base-coco")

c:\Users\sunny\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
pip install safetensors

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import json
import torch

In [ ]:
# Load the JSON file
with open("C:\\Users\\sunny\\Desktop\\research\\cogeval\\dev-bench\\shikra-7b-delta-v1\\pytorch_model.bin.index.json") as f:
    weight_map = json.load(f)["weight_map"]

# Create an empty model
model = torch.nn.Module()

# Load the model state dict
state_dict = {}
for key, value in weight_map.items():
    # Load the weight tensor from the corresponding file
    weight_path = "C:\\Users\\sunny\\Desktop\\research\\cogeval\\dev-bench\\shikra-7b-delta-v1\\" + value
    state_dict[key] = torch.load(weight_path, map_location=torch.device('cpu'))

# Load the state dict into the model
model.load_state_dict(state_dict)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

: 

In [2]:
# load shikras


from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained("shikras/shikra-7b-delta-v1-0708")
#shikras_processor = AutoProcessor.from_pretrained("shikras/shikra-7b-delta-v1")

c:\Users\sunny\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sunny\.cache\huggingface\hub\models--shikras--shikra-7b-delta-v1-0708. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


ValueError: The checkpoint you are trying to load has model type `shikra` but Transformers does not recognize this architecture. This could be because of an issue with the checkpoint, or because your version of Transformers is out of date.

In [19]:
lwl_ds = data_handling.DevBenchDataset("C:\\Users\\sunny\\Desktop\\research\\cogeval\\dev-bench\\lwl\\images\\")
lwl_dl = data_handling.make_dataloader(lwl_ds)

In [22]:
#lwl_sims = extract_values.get_all_sim_scores(lwl_dl, flava_processor, lambda **x: flava_model(**x).logits_per_image)
lwl_sims = extract_values.get_all_sim_scores_flava(lwl_dl, flava_processor, flava_model)

`input_ids_masked` isn't passed which means MLM loss won't be calculated correctlySetting it to `input_ids` so that model can work. Please pass it if this is unintentional. This is usually OKAY if you are doing inference on unmasked text...
c:\Users\sunny\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\modeling_utils.py:909: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
`input_ids_masked` isn't passed which means MLM loss won't be calculated correctlySetting it to `input_ids` so that model can work. Please pass it if this is unintentional. This is usually OKAY if you are doing inference on unmasked text...
`input_ids_masked` isn't passed which means MLM loss won't be calculated correctlySetting it to `input_ids` so that model can work. Please pass it if this is unintentional. This is usually OKAY if you are doing inference on unmasked text...
`input_ids_masked` isn't passed which means MLM loss won'

In [12]:
os.makedirs("C:\\Users\\sunny\\Desktop\\research\\cogeval\\dev-bench\\lwl\\results", exist_ok=True)

In [24]:
import numpy as np
np.save("C:\\Users\\sunny\\Desktop\\research\\cogeval\\dev-bench\\lwl\\results\\lwl_flava.npy", lwl_sims)

In [25]:
trog_ds = data_handling.DevBenchDataset("C:\\Users\\sunny\\Desktop\\research\\cogeval\\dev-bench\\trog\\")
trog_dl = data_handling.make_dataloader(trog_ds)

In [26]:
#trog_sims = extract_values.get_all_sim_scores(trog_dl, processor, lambda **x: model(**x).logits_per_image)
trog_sims = extract_values.get_all_sim_scores_flava(trog_dl, flava_processor, flava_model)

c:\Users\sunny\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\tokenization_utils_base.py:2436: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
`input_ids_masked` isn't passed which means MLM loss won't be calculated correctlySetting it to `input_ids` so that model can work. Please pass it if this is unintentional. This is usually OKAY if you are doing inference on unmasked text...
c:\Users\sunny\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\modeling_utils.py:909: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
`input_ids_masked` isn't passed which means MLM loss won't be calculated correctlySetting it to `input_ids` so that model can work. Please pass it if this is unintentional. This is usually OKAY if you are doing inference on unmasked text...
`input_ids_maske

In [29]:
import numpy as np
np.save("C:\\Users\\sunny\\Desktop\\research\\cogeval\\dev-bench\\eval\\trog\\trog_flava.npy", trog_sims)

In [30]:
winoground_ds = data_handling.DevBenchDataset("C:\\Users\\sunny\\Desktop\\research\\cogeval\\dev-bench\\winoground\\")
winoground_dl = data_handling.make_dataloader(winoground_ds)

In [31]:
#winoground_sims = extract_values.get_all_sim_scores(winoground_dl, processor, lambda **x: model(**x).logits_per_image)
winoground_sims = extract_values.get_all_sim_scores_flava(winoground_dl, flava_processor, flava_model)

c:\Users\sunny\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\tokenization_utils_base.py:2436: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
`input_ids_masked` isn't passed which means MLM loss won't be calculated correctlySetting it to `input_ids` so that model can work. Please pass it if this is unintentional. This is usually OKAY if you are doing inference on unmasked text...
c:\Users\sunny\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\modeling_utils.py:909: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
`input_ids_masked` isn't passed which means MLM loss won't be calculated correctlySetting it to `input_ids` so that model can work. Please pass it if this is unintentional. This is usually OKAY if you are doing inference on unmasked text...
`input_ids_maske

In [7]:
os.makedirs("C:\\Users\\sunny\\Desktop\\research\\cogeval\\dev-bench\\winground\\results", exist_ok=True)

In [33]:
import numpy as np
np.save("C:\\Users\\sunny\\Desktop\\research\\cogeval\\dev-bench\\eval\\winoground\\winground_flava.npy", winoground_sims)

In [34]:
vizvocab_ds = data_handling.DevBenchDataset("C:\\Users\\sunny\\Desktop\\research\\cogeval\\dev-bench\\viz_vocab\\")
vizvocab_dl = data_handling.make_dataloader(vizvocab_ds)

In [36]:
#vizvocab_sims = extract_values.get_all_sim_scores(vizvocab_dl, processor, lambda **x: model(**x).logits_per_image)
vizvocab_sims = extract_values.get_all_sim_scores_flava(vizvocab_dl, flava_processor, flava_model)

`input_ids_masked` isn't passed which means MLM loss won't be calculated correctlySetting it to `input_ids` so that model can work. Please pass it if this is unintentional. This is usually OKAY if you are doing inference on unmasked text...
`input_ids_masked` isn't passed which means MLM loss won't be calculated correctlySetting it to `input_ids` so that model can work. Please pass it if this is unintentional. This is usually OKAY if you are doing inference on unmasked text...
`input_ids_masked` isn't passed which means MLM loss won't be calculated correctlySetting it to `input_ids` so that model can work. Please pass it if this is unintentional. This is usually OKAY if you are doing inference on unmasked text...
`input_ids_masked` isn't passed which means MLM loss won't be calculated correctlySetting it to `input_ids` so that model can work. Please pass it if this is unintentional. This is usually OKAY if you are doing inference on unmasked text...
`input_ids_masked` isn't passed whic

In [14]:
os.makedirs("C:\\Users\\sunny\\Desktop\\research\\cogeval\\dev-bench\\vizvocab\\results", exist_ok=True)

In [39]:
np.save("C:\\Users\\sunny\\Desktop\\research\\cogeval\\dev-bench\\eval\\visual_vocab\\vizvocab_flava.npy", vizvocab_sims)

In [40]:
aro_ds = data_handling.DevBenchDataset("C:\\Users\\sunny\\Desktop\\research\\cogeval\\dev-bench\\aro\\")
aro_dl = data_handling.make_dataloader(aro_ds)

In [46]:
#aro_sims = extract_values.get_all_sim_scores(aro_dl, processor, lambda **x: model(**x).logits_per_image)


aro_sims = extract_values.get_all_sim_scores_flava_aro(aro_dl, flava_processor, flava_model)

`input_ids_masked` isn't passed which means MLM loss won't be calculated correctlySetting it to `input_ids` so that model can work. Please pass it if this is unintentional. This is usually OKAY if you are doing inference on unmasked text...
`input_ids_masked` isn't passed which means MLM loss won't be calculated correctlySetting it to `input_ids` so that model can work. Please pass it if this is unintentional. This is usually OKAY if you are doing inference on unmasked text...
`input_ids_masked` isn't passed which means MLM loss won't be calculated correctlySetting it to `input_ids` so that model can work. Please pass it if this is unintentional. This is usually OKAY if you are doing inference on unmasked text...
`input_ids_masked` isn't passed which means MLM loss won't be calculated correctlySetting it to `input_ids` so that model can work. Please pass it if this is unintentional. This is usually OKAY if you are doing inference on unmasked text...
`input_ids_masked` isn't passed whic

KeyboardInterrupt: 

In [14]:
len(aro_sims)

9833

In [15]:
os.makedirs("C:\\Users\\sunny\\Desktop\\research\\cogeval\\dev-bench\\aro\\results", exist_ok=True)

In [16]:
np.save("C:\\Users\\sunny\\Desktop\\research\\cogeval\\dev-bench\\aro\\results\\aro.npy", aro_sims)

In [17]:
voc_ds = data_handling.DevBenchDataset("assets/sem-viz_obj_cat/")
voc_dl = data_handling.make_dataloader(voc_ds)

In [18]:
voc_embeds = extract_values.get_all_image_feats(voc_dl, processor, model.get_image_features)

In [21]:
men_ds = data_handling.DevBenchDataset("C:\\Users\\sunny\\Desktop\\research\\cogeval\\dev-bench\\men\\")
men_dl = data_handling.make_dataloader(men_ds)

In [29]:
import torch
from tqdm import tqdm

In [30]:
def get_text_features(texts):
    with torch.no_grad():
        inputs = blip_processor(text=texts, padding=True, truncation=True, return_tensors="pt")
        outputs = blip_model.text_encoder(**inputs)
        text_features = outputs.last_hidden_state[:, 0, :]  # Extract the CLS token embedding
    return text_features

In [33]:
#men_embeds = extract_values.get_all_text_feats(men_dl, flava_processor, flava_model.get_text_features)
#men_embeds = extract_values.get_all_text_feats(men_dl, flava_processor, flava_model.text_encoder)

# embedding for flava
men_embeds = extract_values.get_all_text_feats_flava(men_dl, flava_processor, flava_model)


# embedding for blip
men_embeds = extract_values.get_all_text_feats_blip(men_dl, blip_processor, get_text_features)


Extracting text features: 100%|██████████| 47/47 [00:06<00:00,  7.64it/s]


In [35]:
men_embeds

tensor([[ 0.0625,  0.0207, -0.0566,  ..., -0.0254, -0.0360, -0.0915],
        [-0.0409, -0.0147, -0.0311,  ...,  0.0120,  0.0141,  0.0805],
        [-0.0375, -0.0177, -0.1003,  ...,  0.0121,  0.0262,  0.0756],
        ...,
        [-0.0997, -0.0201,  0.0089,  ..., -0.0048, -0.0434,  0.0973],
        [-0.0398, -0.0110, -0.0395,  ..., -0.0637,  0.0347, -0.0573],
        [-0.0452, -0.0284,  0.0366,  ..., -0.0636, -0.0207,  0.0568]])

In [9]:
os.makedirs("C:\\Users\\sunny\\Desktop\\research\\cogeval\\dev-bench\\men\\results", exist_ok=True)

In [36]:
np.save("C:\\Users\\sunny\\Desktop\\research\\cogeval\\dev-bench\\eval\\men\\men_blip.npy", men_embeds)